In [22]:
import numpy as np
import scipy.io as sio
from sklearn.tree import DecisionTreeRegressor
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.util.reference_direction import UniformReferenceDirectionFactory

# Step 1: Load the data from the .mat file
data = sio.loadmat('training_data.mat')
X_train = data['X_train']  # Features
Y_train = data['Y_train']  # Targets

# Step 2: Train a Decision Tree model
dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, Y_train)

# Step 3: Define the MLDTLZ problem class
class MLDTLZ(Problem):
    def __init__(self, dt_model, **kwargs):
        super().__init__(n_var=8, n_obj=5, n_constr=2, xl=[1, 1e5, 0.1e5, 10, 0.1, 0.01, 0, 1], 
                         xu=[7, 10e5, 0.5e5, 1000, 2, 0.99, 1, 16], **kwargs)
        self.dt_model = dt_model

    def _evaluate(self, X, out, *args, **kwargs):
        # Call the trained decision tree model to make predictions
        Y_pred = self.dt_model.predict(X)

        # Extract individual output predictions
        purity = Y_pred[:, 0]
        recovery = Y_pred[:, 1]
        productivity = Y_pred[:, 2]
        energy_requirement = Y_pred[:, 3]  # Minimize energy requirement
        TCR = Y_pred[:, 4]  # Minimize TCR

        # Assign objectives to the output dictionary
        out["F"] = np.column_stack([purity, recovery, productivity, energy_requirement, TCR])

        # Calculate constraints
        constraint1 = - (0.9 - purity)  # Purity constraint
        constraint2 = - (0.9 - recovery)  # Recovery constraint

        # Assign constraints to the output dictionary
        out["G"] = np.column_stack([constraint1, constraint2])
# Step 4: Create the MLDTLZ problem instance
problem = MLDTLZ(dt_model)

# Step 5: Define the number of reference directions
n_obj = problem.n_obj

# Step 6: Create the reference directions using UniformReferenceDirectionFactory
ref_dirs = UniformReferenceDirectionFactory(n_obj, n_points=70).do()

# Step 7: Initialize the NSGA3 algorithm with reference directions
algorithm = NSGA3(ref_dirs=ref_dirs)

# Step 8: Execute the optimization
res = minimize(problem,
               algorithm,
               seed=1,
               termination=('n_gen', 600))

# Step 9: Visualize the Pareto front
Scatter().add(res.F).show()

# Step 10: Print the optimized decision variables and corresponding outputs
print("Optimized Decision Variables:")
print(res.X)
print("\nCorresponding Outputs:")
print(res.F)


Optimized Decision Variables:
[[3.94475613e+00 7.00957132e+05 3.56431761e+04 5.42929200e+02
  1.60261739e+00 6.13803718e-02 9.66263481e-01 1.40331713e+01]
 [1.67037359e+00 3.59513668e+05 4.92786071e+04 9.32797771e+02
  1.89659095e+00 8.31967682e-01 7.85811574e-01 4.04920562e+00]
 [2.18146953e+00 1.73241197e+05 4.91664841e+04 5.20375972e+02
  1.60256197e+00 9.02293243e-01 8.52514280e-01 2.31331066e+00]
 [1.71172159e+00 8.07813134e+05 4.93093392e+04 4.50617929e+02
  1.84047369e+00 3.93933310e-02 9.66674745e-01 1.38781110e+01]
 [4.33555753e+00 7.90019847e+05 3.73842883e+04 5.14635471e+02
  1.84017897e+00 7.67972455e-01 8.66666938e-01 4.20441014e+00]
 [4.05674007e+00 7.03169477e+05 3.60360183e+04 4.22596832e+02
  1.92604484e+00 3.92573525e-01 9.66506434e-01 1.40331713e+01]
 [6.73459497e+00 7.35839596e+05 4.09475733e+04 8.55776806e+02
  1.82821723e+00 3.23100815e-02 8.52309648e-01 3.91873170e+00]
 [3.78133128e+00 1.38465408e+05 3.76728034e+04 3.08538633e+02
  1.85005232e+00 8.32948003e-02 9